In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gpflowmodi.volumes.strmesh as vol
import scipy.io as sio
import tensorflow as tf
import gpflowmodi
import trimesh

xfilename='data/curvature/'
gtfilename='data/gt/'
surface1='cat0'
surface2='cat1'

gpmethod='vmgplvm' #mvmgplvm-vmgplvm-bvmgplvm
select_init='pca' #bgplvm-pca
itera=1000
num_clusters=3
gerror=False
num_inducing=20
latent_dim=2
learning_rate=1e-1

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
filename=gtfilename+surface1+'-'+surface2+'.mat'
info=sio.whosmat(filename)[0]
mat=sio.loadmat(filename)
gt=mat[info[0]]-1

In [3]:
filename=xfilename+surface1+'-'+surface2+'.mat'
info=sio.whosmat(filename)[0]
mat=sio.loadmat(filename)
X=mat[info[0]]

In [4]:
#Dir data
base='data/nonrigid3d/'
data=surface1+'.mat'
mesh1=vol.StrMesh(filename=base+data)

!Mesh load complete!
Vertices: 5000
Triangles: 9996


In [5]:
base='data/nonrigid3d/'
data=surface2+'.mat'
mesh2=vol.StrMesh(filename=base+data)

!Mesh load complete!
Vertices: 5000
Triangles: 9996


In [7]:
model=gpflowmodi.models.RunGPLVM(gpmethod=gpmethod, itera=itera, learning_rate=learning_rate, select_init=select_init, num_clusters=num_clusters, verbose=True, num_inducing=num_inducing, latent_dim=latent_dim).fit(X)
if gpmethod=='mvmgplvm':
    Z=np.argmax(tf.gather(tf.concat([model.model.gamma0,model.model.gamma1],axis=1),model.order_indices,axis=1).numpy(),axis=0)
else:
    Z=np.argmax(model.model.gamma.numpy(),axis=0)

Iteration: 0
Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100
Iteration: 110
Iteration: 120
Iteration: 130
Iteration: 140
Iteration: 150
Iteration: 160
Iteration: 170
Iteration: 180
Iteration: 190
Iteration: 200
Iteration: 210
Iteration: 220
Iteration: 230
Iteration: 240
Iteration: 250
Iteration: 260
Iteration: 270
Iteration: 280
Iteration: 290
Iteration: 300
Iteration: 310
Iteration: 320
Iteration: 330
Iteration: 340
Iteration: 350
Iteration: 360
Iteration: 370
Iteration: 380
Iteration: 390
Iteration: 400
Iteration: 410
Iteration: 420
Iteration: 430
Iteration: 440
Iteration: 450
Iteration: 460
Iteration: 470
Iteration: 480
Iteration: 490
Iteration: 500
Iteration: 510
Iteration: 520
Iteration: 530
Iteration: 540
Iteration: 550
Iteration: 560
Iteration: 570
Iteration: 580
Iteration: 590
Iteration: 600
Iteration: 610
Iteration: 620
Iteration: 630
Iteration: 640
Iteration: 650
Iteration: 660
Iterat

In [12]:
Z=np.argmax(Z,axis=0)

In [14]:
tam1=mesh1.vertices.shape[0]
tam2=mesh2.vertices.shape[0]
Z1=Z[0:tam1]
Z2=Z[tam1:]

In [9]:
norm=np.sqrt(mesh2.getArea())
print(norm)

AttributeError: 'Trimesh' object has no attribute 'get_surface_area'

In [ ]:
import gdist
if gerror:
    vertices=mesh2.vertices.astype(np.float64)
    triangles = mesh2.triangles.astype(np.int32)
    geodist=np.array([])
    for i in range(tam1):
        if not(np.isnan(gt[i])):
            print('Vertex number:',i)
            label=Z1[i]
            src=np.array(gt[i],dtype=np.int32)
            trg=np.where(Z2==label)[0].astype(np.int32)
            dist=gdist.compute_gdist(vertices, triangles, source_indices=src, target_indices=trg)
            geodist=np.concatenate((geodist,dist.min(keepdims=True)))#dist.min(keepdims=True)
    geodist=geodist/norm

    step=0.005
    thresh = np.arange(0,0.25+step,step)
    curve=[]
    for i in range(thresh.shape[0]):
        curve.append(100*np.sum(geodist<=thresh[i])/geodist.shape[0])
    curve=np.array(curve)

    plt.plot(thresh,curve)
    plt.show()

    index=np.where(curve<=100)
    percentage=np.max(curve[index])
    ind=np.where(curve==percentage)[0]
    error=np.min(thresh[ind])
    print('Geodesic Error:',error)
    print('Percentage:',percentage)

Vertex number: 3484
Vertex number: 3487
Vertex number: 3488
Vertex number: 3489
Vertex number: 3490
Vertex number: 3492
Vertex number: 3494
Vertex number: 3496
Vertex number: 3498
Vertex number: 3501
Vertex number: 3502
Vertex number: 3503
Vertex number: 3505
Vertex number: 3506
Vertex number: 3507
Vertex number: 3508
Vertex number: 3510
Vertex number: 3512
Vertex number: 3515
Vertex number: 3516
Vertex number: 3519
Vertex number: 3520
Vertex number: 3522
Vertex number: 3524
Vertex number: 3525
Vertex number: 3528
Vertex number: 3529
Vertex number: 3531
Vertex number: 3533
Vertex number: 3534
Vertex number: 3537
Vertex number: 3538
Vertex number: 3539
Vertex number: 3543
Vertex number: 3544
Vertex number: 3547
Vertex number: 3551
Vertex number: 3557
Vertex number: 3559
Vertex number: 3562
Vertex number: 3563
Vertex number: 3564
Vertex number: 3566
Vertex number: 3568
Vertex number: 3570
Vertex number: 3575
Vertex number: 3577
Vertex number: 3579
Vertex number: 3583
Vertex number: 3586


In [15]:
rmin=np.min(Z1)
rmax=np.max(Z1)
mesh1.assignHeatColor(minimum=rmin,maximum=rmax,value=Z1)
rmin=np.min(Z2)
rmax=np.max(Z2)
mesh2.assignHeatColor(minimum=rmin,maximum=rmax,value=Z2)

In [25]:
mesh1.mesh.apply_translation([0,0,0])
mesh2.mesh.apply_translation([-100,0,0])
scene=trimesh.Scene([mesh1.mesh,mesh2.mesh])
scene.show()

In [22]:
import tempfile
with tempfile.NamedTemporaryFile(suffix='.png') as file_obj:
    scene.save_image(file_obj.name)
    file_obj.seek(0)
    data = file_obj.read()

ValueError: invalid literal for int() with base 10: 'C:\\Users\\minol\\AppData\\Local\\Temp\\tmpz93eawlk.png'